In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords # the for of in with
from nltk.stem.porter import PorterStemmer # loved loving == love
from sklearn.feature_extraction.text import TfidfVectorizer # loved = [0.0]
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
news_df = pd.read_csv('/content/drive/MyDrive/Dataset fake_news/train.csv')

In [ ]:
news_df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
news_df.shape

(20800, 5)

In [ ]:
news_df.isna().sum()

,0
id,0
title,558
author,1957
text,39
label,0


In [ ]:
news_df = news_df.fillna(' ')

In [ ]:
news_df.isna().sum()

,0
id,0
title,0
author,0
text,0
label,0


In [ ]:
#news_df['content'] = news_df['author']+" "+news_df['title']+" "+news_df['text']
news_df['content'] = news_df['title']+" "+news_df['text']

In [ ]:
news_df.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"FLYNN: Hillary Clinton, Big Woman on Campus - ..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Why the Truth Might Get You Fired Why the Trut...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,15 Civilians Killed In Single US Airstrike Hav...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Iranian woman jailed for fictional unpublished...


In [ ]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# stemming
ps = PorterStemmer()
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [ps.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [ ]:
!pip install swifter


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 54.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.2/243.2 kB 18.7 MB/s eta 0:00:00
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16507 sha256=155ce0753f805a60d31234620187e960c62b2f561c187a9f8e0c56e7b746e97a
  Stored in directory: /root/.cache/pip/wheels/e4/cf/51/0904952972ee2c7aa3709437065278dc534ec1b8d2ad41b443
Successfully built swifter


In [ ]:
import swifter
news_df['content'] = news_df['content'].swifter.apply(stemming)


Pandas Apply:   0%|          | 0/20800 [00:00<?, ?it/s]

In [ ]:
#news_df['content'] = news_df['content'].apply(stemming)

In [ ]:
news_df['content']

,content
0,hous dem aid even see comey letter jason chaff...
1,flynn hillari clinton big woman campu breitbar...
2,truth might get fire truth might get fire octo...
3,civilian kill singl us airstrik identifi video...
4,iranian woman jail fiction unpublish stori wom...
...,...
20795,rapper trump poster child white supremaci rapp...
20796,n f l playoff schedul matchup odd new york tim...
20797,maci said receiv takeov approach hudson bay ne...
20798,nato russia hold parallel exercis balkan nato ...


In [ ]:
X = news_df['content'].values
y = news_df['label'].values
print(X)

['hous dem aid even see comey letter jason chaffetz tweet hous dem aid even see comey letter jason chaffetz tweet darrel lucu octob subscrib jason chaffetz stump american fork utah imag courtesi michael jolley avail creativ common licens apolog keith olbermann doubt worst person world week fbi director jame comey accord hous democrat aid look like also know second worst person well turn comey sent infam letter announc fbi look email may relat hillari clinton email server rank democrat relev committe hear comey found via tweet one republican committe chairmen know comey notifi republican chairmen democrat rank member hous intellig judiciari oversight committe agenc review email recent discov order see contain classifi inform long letter went oversight committe chairman jason chaffetz set polit world ablaz tweet fbi dir inform fbi learn exist email appear pertin investig case reopen jason chaffetz jasoninthehous octob cours know case comey actual say review email light unrel case know an

In [ ]:
vector = TfidfVectorizer()
vector.fit(X)
X = vector.transform(X)


In [ ]:
import joblib
joblib.dump(vector, "vectorizer.pkl")


['vectorizer.pkl']

In [ ]:
from google.colab import files
files.download('vectorizer.pkl')



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print(X)

  (0, 323)	0.044179022507208135
  (0, 518)	0.02332405077764721
  (0, 632)	0.039316203504174516
  (0, 869)	0.015976541921521512
  (0, 918)	0.016642095819834255
  (0, 1292)	0.0213987579984195
  (0, 1602)	0.018255643322868005
  (0, 1877)	0.11720929349150563
  (0, 3008)	0.04733009098564867
  (0, 3028)	0.019311970062028613
  (0, 3363)	0.011979048895968012
  (0, 3725)	0.03134889384323141
  (0, 4135)	0.016933166314925904
  (0, 4237)	0.01971670091155046
  (0, 4282)	0.026932122951737247
  (0, 4563)	0.017964895769890684
  (0, 4746)	0.026333626062969923
  (0, 4796)	0.04231759693995469
  (0, 4813)	0.01462055924601113
  (0, 6387)	0.018502204351742796
  (0, 6930)	0.0209635900182605
  (0, 8509)	0.021972639591348223
  (0, 9022)	0.014453700828335692
  (0, 10780)	0.04830805844960349
  (0, 12171)	0.02820114316501808
  :	:
  (20799, 106136)	0.03677728368716433
  (20799, 106269)	0.04946563012472331
  (20799, 106585)	0.018756039260858177
  (20799, 106619)	0.04336474130063181
  (20799, 106943)	0.017495757820

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y, random_state=1)

In [ ]:
model = LogisticRegression()
model.fit(X_train,y_train)

LogisticRegression()

In [ ]:
train_y_pred = model.predict(X_train)
train_accuracy = accuracy_score(train_y_pred,y_train)
print(train_accuracy)

0.9745192307692307


In [ ]:
test_y_pred = model.predict(X_test)
test_accuracy = accuracy_score(test_y_pred,y_test)
print(test_accuracy)

0.9512019230769231


In [ ]:
# using random forest
from sklearn.ensemble import RandomForestClassifier
model_rf = RandomForestClassifier(n_estimators=100, random_state=42)
model_rf.fit(X_train, y_train)

train_y_pred = model_rf.predict(X_train)
train_accuracy = accuracy_score(train_y_pred,y_train)
print(train_accuracy)

test_y_pred = model_rf.predict(X_test)
test_accuracy = accuracy_score(test_y_pred,y_test)
print(test_accuracy)



1.0
0.9242788461538461


In [ ]:
#using linearsvc
from sklearn.svm import LinearSVC
model_linear = LinearSVC()
model_linear.fit(X_train, y_train)

train_y_pred = model_linear.predict(X_train)
train_accuracy = accuracy_score(train_y_pred,y_train)
print(train_accuracy)

test_y_pred = model_linear.predict(X_test)
test_accuracy = accuracy_score(test_y_pred,y_test)
print(test_accuracy)

0.9990384615384615
0.9665865384615384


In [ ]:
#using xgboost
from xgboost import XGBClassifier
model_xgboost = XGBClassifier()
model_xgboost.fit(X_train, y_train)

train_y_pred = model_xgboost.predict(X_train)
train_accuracy = accuracy_score(train_y_pred,y_train)
print(train_accuracy)

test_y_pred = model_xgboost.predict(X_test)
test_accuracy = accuracy_score(test_y_pred,y_test)
print(test_accuracy)

0.9998798076923077
0.9762019230769231


In [ ]:
# prediction system

input_data = X_test[1]
prediction = model.predict(input_data)
if prediction[0] == 1:
    print('Fake news')
else:
    print('Real news')

Real news


In [ ]:
def predict_news_from_input():
    """Predicts whether the given news is fake or real based on user-provided title and author."""

    # Take input directly within the function
    title = input("Enter the news title: ")
    #author = input("Enter the author's name: ")
    news_text = input("Enter the news text: ")

    # Combine title and author to match the training data format
    combined_input = title + " "+news_text

    # Preprocess the combined input (e.g., stemming, cleaning)
    stemmed_content = stemming(combined_input)

    # Transform the input using the vectorizer used for training
    input_data = vector.transform([stemmed_content])

    # Make the prediction
    prediction = model_xgboost.predict(input_data)

    # Return the prediction result
    if prediction[0] == 1:
        print("Fake news")
    else:
        print("Real news")

# Run the prediction function
predict_news_from_input()


Enter the news title: South Korea President Yoon Suk Yeol impeached over his martial law order
Enter the news text: The National Assembly passed the motion in a 204-85 vote on Saturday, December 14, 2024. Mr. Yoon’s Presidential powers and duties will be suspended after the copies of a document on the impeachment are delivered to him and to the Constitutional Court. The court has up to 180 days to determine whether to dismiss Yoon as president or restore his powers. If he’s thrown out of office, a national election to choose his successor must be held within 60 days.
Fake news


In [ ]:
import joblib
joblib.dump(model_xgboost, 'fake_news_model.pkl')


In [ ]:
from google.colab import files
files.download('fake_news_model.pkl')

